In [1]:
! pip install torchfm

  Created wheel for torchfm: filename=torchfm-0.7.0-py3-none-any.whl size=18353 sha256=d716a705fd990c397629800b3c9317c2aea590c87c3778ee330dc000f059b618
  Stored in directory: /Users/apple/Library/Caches/pip/wheels/5e/e9/25/1ae407681ff67b4334513b7793ced4c66a4bf37ee99ed31ffe
Successfully built torchfm


In [21]:
import torch
import tqdm
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader
import os
import pandas as pd

from torchfm.model.afi import AutomaticFeatureInteractionModel
from torchfm.model.afm import AttentionalFactorizationMachineModel
from torchfm.model.dcn import DeepCrossNetworkModel
from torchfm.model.dfm import DeepFactorizationMachineModel
from torchfm.model.ffm import FieldAwareFactorizationMachineModel
from torchfm.model.fm import FactorizationMachineModel
from torchfm.model.fnfm import FieldAwareNeuralFactorizationMachineModel
from torchfm.model.fnn import FactorizationSupportedNeuralNetworkModel
from torchfm.model.lr import LogisticRegressionModel
from torchfm.model.ncf import NeuralCollaborativeFiltering
from torchfm.model.nfm import NeuralFactorizationMachineModel
from torchfm.model.pnn import ProductNeuralNetworkModel
from torchfm.model.wd import WideAndDeepModel
from torchfm.model.xdfm import ExtremeDeepFactorizationMachineModel
from torchfm.model.afn import AdaptiveFactorizationNetwork

In [22]:
new_df = pd.read_csv(os.path.join("..","..","data","input_data.csv"))
new_df.head()

/Users/apple/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (6,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,idd,user.id,place.id,place.name,category,place.category,user.rating,place.rating,createdDate,is_fch,photonum,review.count,language,lat,lng,land.addr,review,is_local
0,0,0,4TXey,1.150710e+09,스타벅스 보라매공원R점,EAT,스타벅스,4.5,4.590717,2020-05-21 20:42:00,1,0.0,158,ko,37.492103,126.923551,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",NaN,1
1,1,1,awWC,1.150710e+09,스타벅스 보라매공원R점,EAT,스타벅스,5.0,4.590717,2019-06-28 01:11:03,1,0.0,158,ko,37.492103,126.923551,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",none,1
2,2,2,1Unzg,1.150710e+09,스타벅스 보라매공원R점,EAT,스타벅스,5.0,4.590717,2020-03-12 00:22:00,1,0.0,158,ko,37.492103,126.923551,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",NaN,1
3,3,3,LRVl,1.150710e+09,스타벅스 보라매공원R점,EAT,스타벅스,5.0,4.590717,2020-02-16 17:06:00,1,0.0,158,ko,37.492103,126.923551,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",NaN,1
4,4,4,ESN1,1.150710e+09,스타벅스 보라매공원R점,EAT,스타벅스,5.0,4.590717,2019-03-11 00:08:48,1,0.0,158,ko,37.492103,126.923551,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",2019.03.06\nHH,1


In [24]:
new_df.drop(columns='idd', axis=1, inplace=True)
new_df.head()

,user.id,place.id,place.name,category,place.category,user.rating,place.rating,createdDate,is_fch,photonum,review.count,language,lat,lng,land.addr,review,is_local
0,4TXey,1.150710e+09,스타벅스 보라매공원R점,EAT,스타벅스,4.5,4.590717,2020-05-21 20:42:00,1,0.0,158,ko,37.492103,126.923551,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",NaN,1
1,awWC,1.150710e+09,스타벅스 보라매공원R점,EAT,스타벅스,5.0,4.590717,2019-06-28 01:11:03,1,0.0,158,ko,37.492103,126.923551,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",none,1
2,1Unzg,1.150710e+09,스타벅스 보라매공원R점,EAT,스타벅스,5.0,4.590717,2020-03-12 00:22:00,1,0.0,158,ko,37.492103,126.923551,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",NaN,1
3,LRVl,1.150710e+09,스타벅스 보라매공원R점,EAT,스타벅스,5.0,4.590717,2020-02-16 17:06:00,1,0.0,158,ko,37.492103,126.923551,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",NaN,1
4,ESN1,1.150710e+09,스타벅스 보라매공원R점,EAT,스타벅스,5.0,4.590717,2019-03-11 00:08:48,1,0.0,158,ko,37.492103,126.923551,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",2019.03.06\nHH,1


In [ ]:
new_df.to_csv(os.path.join("..","..","data","input_data.csv"), )

In [16]:
def get_dataset(name):
    if name == 'TripAdvisor':
        new_df = pd.read_csv(os.path.join("..","..","data","TA_User_Reviws_Korea_all_v2_new_df.csv"))
        new_df = new_df.drop(['location_name'], axis=1)
        return new_df
    elif name == "FinalData":
        new_df = pd.read_csv(os.path.join("..","..","data","input_data.csv"))
#         new_df = new_df.drop(['location_name'], axis=1)
        return new_df
    else:
        raise ValueError('unknown dataset name: ' + name)

In [17]:
def get_model(name, dataset):
    """
    Hyperparameters are empirically determined, not opitmized.
    """
    field_dims = dataset.field_dims
    if name == 'lr':
        return LogisticRegressionModel(field_dims)
    elif name == 'fm':
        return FactorizationMachineModel(field_dims, embed_dim=16)
    elif name == 'ffm':
        return FieldAwareFactorizationMachineModel(field_dims, embed_dim=4)
    elif name == 'fnn':
        return FactorizationSupportedNeuralNetworkModel(field_dims, embed_dim=16, mlp_dims=(16, 16), dropout=0.2)
    elif name == 'wd':
        return WideAndDeepModel(field_dims, embed_dim=16, mlp_dims=(16, 16), dropout=0.2)
    elif name == 'ipnn':
        return ProductNeuralNetworkModel(field_dims, embed_dim=16, mlp_dims=(16,), method='inner', dropout=0.2)
    elif name == 'opnn':
        return ProductNeuralNetworkModel(field_dims, embed_dim=16, mlp_dims=(16,), method='outer', dropout=0.2)
    elif name == 'dcn':
        return DeepCrossNetworkModel(field_dims, embed_dim=16, num_layers=3, mlp_dims=(16, 16), dropout=0.2)
    elif name == 'nfm':
        return NeuralFactorizationMachineModel(field_dims, embed_dim=64, mlp_dims=(64,), dropouts=(0.2, 0.2))
    elif name == 'ncf':
        # only supports MovieLens dataset because for other datasets user/item colums are indistinguishable
        assert isinstance(dataset, MovieLens20MDataset) or isinstance(dataset, MovieLens1MDataset)
        return NeuralCollaborativeFiltering(field_dims, embed_dim=16, mlp_dims=(16, 16), dropout=0.2,
                                            user_field_idx=dataset.user_field_idx,
                                            item_field_idx=dataset.item_field_idx)
    elif name == 'fnfm':
        return FieldAwareNeuralFactorizationMachineModel(field_dims, embed_dim=4, mlp_dims=(64,), dropouts=(0.2, 0.2))
    elif name == 'dfm':
        return DeepFactorizationMachineModel(field_dims, embed_dim=16, mlp_dims=(16, 16), dropout=0.2)
    elif name == 'xdfm':
        return ExtremeDeepFactorizationMachineModel(
            field_dims, embed_dim=16, cross_layer_sizes=(16, 16), split_half=False, mlp_dims=(16, 16), dropout=0.2)
    elif name == 'afm':
        return AttentionalFactorizationMachineModel(field_dims, embed_dim=16, attn_size=16, dropouts=(0.2, 0.2))
    elif name == 'afi':
        return AutomaticFeatureInteractionModel(
             field_dims, embed_dim=16, atten_embed_dim=64, num_heads=2, num_layers=3, mlp_dims=(400, 400), dropouts=(0, 0, 0))
    elif name == 'afn':
        print("Model:AFN")
        return AdaptiveFactorizationNetwork(
            field_dims, embed_dim=16, LNN_dim=1500, mlp_dims=(400,400,400), dropouts=(0, 0, 0))
    else:
        raise ValueError('unknown model name: ' + name)

In [18]:
def train(model, optimizer, data_loader, criterion, device, log_interval=1000):
    model.train()
    total_loss = 0
    for i, (fields, target) in enumerate(tqdm.tqdm(data_loader, smoothing=0, mininterval=1.0)):
        fields, target = fields.to(device), target.to(device)
        y = model(fields)
        loss = criterion(y, target.float())
        model.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if (i + 1) % log_interval == 0:
            print('    - loss:', total_loss / log_interval)
            total_loss = 0

In [8]:
# def test(model, data_loader, device):
#     model.eval()
#     targets, predicts = list(), list()
#     with torch.no_grad():
#         for fields, target in tqdm.tqdm(data_loader, smoothing=0, mininterval=1.0):
#             fields, target = fields.to(device), target.to(device)
#             y = model(fields)
#             targets.extend(target.tolist())
#             predicts.extend(y.tolist())
#     return roc_auc_score(targets, predicts)

In [19]:
def main(dataset_name,
         model_name,
         epoch,
         learning_rate,
         batch_size,
         weight_decay,
         device,
         save_dir):
    device = torch.device(device)
    dataset = get_dataset(dataset_name)
    train_length = int(len(dataset) * 0.95)
    valid_length = int(len(dataset) * 0.05)
#     test_length = len(dataset) - train_length - valid_length
    train_dataset, valid_dataset = torch.utils.data.random_split(
        dataset, (train_length, valid_length))
    train_data_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=8)
    valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=8)
#     test_data_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=8)
    model = get_model(model_name, dataset).to(device)
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.RMSprop(params=model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    for epoch_i in range(epoch):
        train(model, optimizer, train_data_loader, criterion, device)
#         auc = test(model, valid_data_loader, device)
        print('epoch:', epoch_i, 'validation: auc:', auc)
#     auc = test(model, test_data_loader, device)
#     print('test auc:', auc)
#     torch.save(model, f'{save_dir}/{model_name}.pt')

In [14]:
if __name__ == '__main__':
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument('--dataset_name', default='criteo')
    parser.add_argument('--model_name', default='afm')
    parser.add_argument('--epoch', type=int, default=15)
    parser.add_argument('--learning_rate', type=float, default=0.001)
    parser.add_argument('--batch_size', type=int, default=2048)
    parser.add_argument('--weight_decay', type=float, default=1e-6)
    parser.add_argument('--device', default='cuda:0')
#     parser.add_argument('--save_dir', default='chkpt')
    args = parser.parse_args()
    main(args.dataset_name,
         args.model_name,
         args.epoch,
         args.learning_rate,
         args.batch_size,
         args.weight_decay,
         args.device,
         args.save_dir)

usage: ipykernel_launcher.py [-h] [--dataset_name DATASET_NAME]
                             [--dataset_path DATASET_PATH]
                             [--model_name MODEL_NAME] [--epoch EPOCH]
                             [--learning_rate LEARNING_RATE]
                             [--batch_size BATCH_SIZE]
                             [--weight_decay WEIGHT_DECAY] [--device DEVICE]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/apple/Library/Jupyter/runtime/kernel-65b22c55-119c-4ad1-a271-245ed9403645.json


SystemExit: 2

/Users/apple/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
